# Cluster Id's based on correlation of NaN structure between Id
# but first
## All NaN are not Random Confirmation
Hypothesis :
- From [Devin's](https://www.kaggle.com/devinanzelmo/two-sigma-financial-modeling/pandas-rolling-and-a-quick-look-at-missing-values) and [Reziproke's](https://www.kaggle.com/reziproke/two-sigma-financial-modeling/nan-structure-on-id-level-eda/notebook) analysis. Taking a look at the missing values, the first thing that pops out is that once valid values occur there are no more missing values

How to check the hypothesis?
- Check wether there are some missing value in the middle

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

pd.set_option('display.max_columns', 120)

In [ ]:
with pd.HDFStore("../input/train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

df.head()

Change NaN to 1. and non-NaN into 0

In [ ]:
features = [x for x in df.columns.values if x not in ['id','y','timestamp']]

In [ ]:
def makeBinaryNaN(x):
    if x == x: # if x == x is true then is not NaN, so return 0
        return 0.
    else: # if NaN return 1
        return 1.
    
dfnan = df.applymap(makeBinaryNaN)
dfnan['id'] = df['id']
dfnan['timestamp'] = df['timestamp']
dfnan.head()

Next, find signal that indicates that there is a change from 0 (non-NaN) to 1 (NaN) across timestamp. To do that I differentiate data with it's shifted form. The sum them. If there are more that 1 signal, it means maybe there exist NaN in the middle.

In [ ]:
ids = dfnan['id'].unique()

In [ ]:
df_agg_signal = pd.DataFrame()
for id in ids:
    df_signal = dfnan[dfnan['id'] == id][features].apply(lambda x: abs(x-x.shift()))
    df_signal.dropna(inplace=True)
    df_agg_signal[id] = df_signal.sum()

df_agg_signal.head()

detect wether there are value > 1.0 in above dataFrame

In [ ]:
listofIdwithMiddleNaN = []

for id in df_agg_signal.columns:
    if ([True] in (df_agg_signal[id].values > 1.)):
        listofIdwithMiddleNaN.append(theid)

In [ ]:
len(listofIdwithMiddleNaN)

So all the values are 0 or 1. I cannot find double changing signal. Either NaN appers at few first row only, or all are NaN, or there is no NaN. No NaN is in the middle. So it confirms [Devin's](https://www.kaggle.com/devinanzelmo) and [Reziproke's](https://www.kaggle.com/reziproke/two-sigma-financial-modeling/nan-structure-on-id-level-eda/notebook) hypothesis that the NaN structure are not random. 

# Next : Cluster Id's based on correlation of NaN structure between Id

As [Reziproke's](https://www.kaggle.com/reziproke/two-sigma-financial-modeling/nan-structure-on-id-level-eda/notebook) and [lesibius's](https://www.kaggle.com/lesibius/two-sigma-financial-modeling/financial-instrument-types) analysis. Can Ids be clustered based of their NaN structure?

Here I try to use NaN ratio as Reziproke and then try to cluster Id based on correlation between Id's NaN correlation

Fist I define counter variable that would be used to help me calculate how much timestamp for each Id

In [ ]:
dfnan['counter'] = [1]*dfnan.shape[0]

The group our nan dataframe by id's and with summation aggregate to find how much NaN appear for each Id

In [ ]:
dfnan_gb = dfnan[['id']+['counter']+features].groupby('id').agg('sum')
dfnan_gb.head()

Next I devide data for each column by the counter column to find relative ratio of NaN. The values are between 0 and 1. If the result is 0 then there are no NaN, but if 1 then all are NaN as follows

In [ ]:
df_rel_nan = dfnan_gb.apply(lambda x: x/x[0], axis=1)
df_rel_nan.head()

The following is the visualization of above dataframe without the counter part

In [ ]:
plt.figure()
sns.heatmap(df_rel_nan[features].as_matrix())

Next, I want calculate the correlation of NaN structure between each Id, to do that, I need to transpose the dataframe as follow

In [ ]:
df_rel_nan_t = df_rel_nan[features].T
df_rel_nan_t.head()

## Correlation of NaN structure between Id
Then calculate the correlation

In [ ]:
corr = df_rel_nan_t.corr()
corr.head()

Here the cluster map visualization of the correlation matrix

In [ ]:
plt.figure(figsize=(14,14))
sns.clustermap(corr.as_matrix())

## Affinity Propagation Clustering

I dont know how much cluster there, so I try to cluster the with **affinity propagation** clustering algo by which can automatically detect how much cluster that exist

In [ ]:
from sklearn import cluster

_, labels = cluster.affinity_propagation(corr.as_matrix())
n_labels = labels.max()
n_clusters = n_labels + 1

Below I find the 116 clusters and their members

In [ ]:
Ids = df_rel_nan.index

for i in range(n_labels + 1):
    print('Cluster {}: {}'.format((i), ', '.join( str(e) for e in list(Ids[labels == i]) )   ))

with the size of the clusters are :

In [ ]:
for i in range(n_labels + 1):
    print('Cluster {}: {}'.format((i), len(list(Ids[labels == i]))))

The optimal number of clusters base on NaN structure from **affinity propagation** algorithm are 116. But as we can see there are some big clusters :

- Cluster 27: 181
- Cluster 61: 126
- Cluster 69: 85
- Cluster 104: 65

So I think we can shrink them to into just 5 or 6 Cluster by using **K-Means** clustering algo as follows

## K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 5

labels_kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(corr.as_matrix())

The following are the member of those 5 clusters

In [ ]:
Ids = df_rel_nan.index

for i in range(n_clusters):
    print('Cluster {}: {}'.format((i), ', '.join( str(e) for e in list(Ids[labels_kmeans == i]) )   ))

In [ ]:
for i in range(n_clusters):
    print('Cluster {}: {}'.format((i), len(list(Ids[labels_kmeans == i]))))

Here are the signature characteristic of each cluster

In [ ]:
for i in range(n_clusters):
    Idclus = list(Ids[labels_kmeans == i])
    dfclus = df_rel_nan[features].loc[Idclus].head()
    stripe = dfclus.sum().values.astype(float)/dfclus.sum().values.max()
    plt.figure(figsize = (10,0.5))
    sns.heatmap([list(stripe)], square=True, yticklabels = False, cbar =False)
    plt.text(7, 1.01,'Cluster {}'.format(str(i)), 
             fontsize=10, horizontalalignment='right', color='green', verticalalignment='bottom')
    plt.text(20, 1.01,'Members : {} ...'.format(', '.join( str(e) for e in list(Ids[labels_kmeans == i])[:10] )),
             fontsize=10, color='red', verticalalignment='bottom')